<a href="https://colab.research.google.com/github/A-l-E-v/ML-Engineer/blob/main/Lesson_4_7_(Maps).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
!pip install wget

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import json
import shapely.geometry
import numpy as np
import wget
from urllib.request import urlopen

Plotly поддерживает два основных типа карт:

1) **Mapbox карты.** Это тайловые (плиточные) карты. Технология тайловых карт пришла из видеоигр: там для экономии ресурсов процессора вместо того, чтобы отрисовывать сразу все объекты в высоком разрешении, были отрисованы разные плитки, которые послойно менялись в зависимости от уровня близости игрока к объекту. Так и тут: Тайловые карты представляют собой набор векторных изображений (или плиток), каждое из которых принадлежит своему слою, или, другими словами, отображает тот или иной уровень детализации объектов в зависимости от изменения масштаба. В этом случае информация о карте содержится в *layout.mapbox*.

2) **Geo карты.** Это контурные карты. В отличие от тайловых, они создаются единожды и задаются набором линий, которые приближаются и отдаляются как простой рисунок и не меняются сами по себе в зависимости от масштаба. В этом случае информация о карте содержится в *layout.geo*

При обсуждении инструментала использования карт мы будем отдельно уточнять, для какого конкретно типа мы делаем обзор.

In [ ]:
# https://community.plotly.com/t/plotly-colours-list/11730/6

def show_named_plotly_colours():
    """
    function to display to user the colours to match plotly's named
    css colours.

    Reference:
        #https://community.plotly.com/t/plotly-colours-list/11730/3

    Returns:
        plotly dataframe with cell colour to match named colour name

    """
    s='''
        aliceblue, antiquewhite, aqua, aquamarine, azure,
        beige, bisque, black, blanchedalmond, blue,
        blueviolet, brown, burlywood, cadetblue,
        chartreuse, chocolate, coral, cornflowerblue,
        cornsilk, crimson, cyan, darkblue, darkcyan,
        darkgoldenrod, darkgray, darkgrey, darkgreen,
        darkkhaki, darkmagenta, darkolivegreen, darkorange,
        darkorchid, darkred, darksalmon, darkseagreen,
        darkslateblue, darkslategray, darkslategrey,
        darkturquoise, darkviolet, deeppink, deepskyblue,
        dimgray, dimgrey, dodgerblue, firebrick,
        floralwhite, forestgreen, fuchsia, gainsboro,
        ghostwhite, gold, goldenrod, gray, grey, green,
        greenyellow, honeydew, hotpink, indianred, indigo,
        ivory, khaki, lavender, lavenderblush, lawngreen,
        lemonchiffon, lightblue, lightcoral, lightcyan,
        lightgoldenrodyellow, lightgray, lightgrey,
        lightgreen, lightpink, lightsalmon, lightseagreen,
        lightskyblue, lightslategray, lightslategrey,
        lightsteelblue, lightyellow, lime, limegreen,
        linen, magenta, maroon, mediumaquamarine,
        mediumblue, mediumorchid, mediumpurple,
        mediumseagreen, mediumslateblue, mediumspringgreen,
        mediumturquoise, mediumvioletred, midnightblue,
        mintcream, mistyrose, moccasin, navajowhite, navy,
        oldlace, olive, olivedrab, orange, orangered,
        orchid, palegoldenrod, palegreen, paleturquoise,
        palevioletred, papayawhip, peachpuff, peru, pink,
        plum, powderblue, purple, red, rosybrown,
        royalblue, saddlebrown, salmon, sandybrown,
        seagreen, seashell, sienna, silver, skyblue,
        slateblue, slategray, slategrey, snow, springgreen,
        steelblue, tan, teal, thistle, tomato, turquoise,
        violet, wheat, white, whitesmoke, yellow,
        yellowgreen
        '''
    li=s.split(',')
    li=[l.replace('\n','') for l in li]
    li=[l.replace(' ','') for l in li]

    import pandas as pd
    import plotly.graph_objects as go

    df=pd.DataFrame.from_dict({'colour': li})
    fig = go.Figure(data=[go.Table(
      header=dict(
        values=["Plotly Named CSS colours"],
        line_color='black', fill_color='white',
        align='center', font=dict(color='black', size=14)
      ),
      cells=dict(
        values=[df.colour],
        line_color=[df.colour], fill_color=[df.colour],
        align='center', font=dict(color='black', size=11)
      ))
    ])

    fig.show()

In [ ]:
show_named_plotly_colours()

## 1. Конфигурация карт

### Geo

**NB!** Все конфигурации, о которых мы будем тут говорить, применимы также и к картам, созданным с помощью Plotly Express

In [ ]:
# hello, world! (в этот раз в прямом смысле)
fig = go.Figure(go.Scattergeo())
fig.update_layout(height=400, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# характеристики карты можно менять с помощью .update_geos()
# давайте добавим все фичи на карту

fig = go.Figure(go.Scattergeo())
fig.update_geos \
(
    resolution=50,
    showcoastlines=True, coastlinecolor="RebeccaPurple",
    showland=True, landcolor="LightGreen",
    showocean=True, oceancolor="navy",
    showlakes=True, lakecolor="aqua",
    showrivers=True, rivercolor="royalblue"
)
fig.update_layout(height=400, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# но можно и отключать, если они нам не нужны

fig = go.Figure(go.Scattergeo())
fig.update_geos \
(
    resolution=50,
    visible=False,
    showrivers=True, rivercolor="royalblue"
)
fig.update_layout(height=400, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# можно показать границы стран

fig = go.Figure(go.Scattergeo())
fig.update_geos(
    visible=False, resolution=50,
    showcountries=True, countrycolor="RebeccaPurple"
)
fig.update_layout(height=400, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

**NB!** Во избежание недопониманий, стоит прояснить, что данные по границам опираются на [Natural Earth Data](https://www.naturalearthdata.com/about/), пункт политики которых гласит:

*Natural Earth Vector draws boundaries of countries according to defacto status. We show who actually controls the situation on the ground.*

Что означает, что границы показаны в пределах **де-факто** контролируемых территорий. Более того, некоторые спорные границы могут быть помечены пунктиром, при этом сплошной линией будут показаны границы общеустоявшиеся.

In [ ]:
# можно показать карту в других проекциях

fig = go.Figure(go.Scattergeo())
fig.update_geos(projection_type="orthographic")
fig.update_layout(height=400, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# можно показать карту в других проекциях

fig = go.Figure(go.Scattergeo())
fig.update_geos(projection_type="natural earth")
fig.update_layout(height=400, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# можно повернуть карту и сократить угол обзора

fig = go.Figure(go.Scattergeo())
fig.update_geos\
(
    center={"lon": -50, "lat": 30}, # точка фокуса
    projection_rotation={"lon": 10, "lat": 50, "roll": 30}, # свойства поворота
    lataxis_range=[-100, 50],
    lonaxis_range=[0, 150] # окно обзора
)
fig.update_layout(height=400, margin={"r": 0,"t": 0,"l": 0,"b": 0})
fig.show()

In [ ]:
# параметр fitbounds задает угол обзора, здесь мы задаем его автоматически на основе построенных данных

fig = px.line_geo(lat=[40, 45, 50, 55], lon=[0, 30, 60, 90])
fig.update_geos(fitbounds="locations")
fig.update_layout(height=400, margin={"r": 0,"t": 0,"l": 0,"b": 0})
fig.show()

In [ ]:
# параметр scope позволяет автоматически выбирать подмножество контуров для отрисовки
# более того, он контролирует параметры projection type, center, roll, а также угол обзора

fig = go.Figure(go.Scattergeo())
fig.update_geos \
(
    visible=False,
    resolution=50,
    scope="north america", # или 'world', 'usa', 'europe', 'asia', 'africa', 'south america'.
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Black", subunitwidth=0.1 # доступно только для "north america" и "usa"
)
fig.update_layout(height=400, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# параметр scope позволяет автоматически выбирать подмножество контуров для отрисовки
# более того, он контролирует параметры projection type, center, roll, а также угол обзора

fig = go.Figure(go.Scattergeo())
fig.update_geos \
(
    visible=False,
    resolution=50,
    scope="usa", # или 'world', 'north america', 'europe', 'asia', 'africa', 'south america'.
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Black", subunitwidth=0.1 # доступно только для "north america" и "usa"
)
fig.update_layout(height=400, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# параметр scope позволяет автоматически выбирать подмножество контуров для отрисовки
# более того, он контролирует параметры projection type, center, roll, а также угол обзора

fig = go.Figure(go.Scattergeo())
fig.update_geos \
(
    visible=False,
    resolution=50,
    scope="europe", # или 'world', 'north america', 'usa', 'asia', 'africa', 'south america'.
    showcountries=True, countrycolor="Black"
)
fig.update_layout(height=400, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# также можно нарисовать сетку

fig = go.Figure(go.Scattergeo())
fig.update_geos(lataxis_showgrid=True, lonaxis_showgrid=True)
fig.update_layout(height=400, margin={"r": 0,"t": 0, "l": 0, "b": 0})
fig.show()

### Mapbox

Слои в картах Mapbox делятся на три вида:

1. layout.mapbox.style определяет нижние слои, так называемую *базовую карту*
2. Различные объекты данных будут построены поверх базовой карты.
3. layout.mapbox.layers - это массив, определяющий слои, которые будут построены поверх объектов данных

Пункты 2 и 3 могут контролироваться с помощью параметра *below*

Слово *mapbox* в названиях различных методов и атрибутов ссылаются на Mapbox GL JS-библиотеку с открытым исходным кодом, интегрированную в питоновскеую версию библиотеки Plotly. Если базовая карта, определенная в layout.mapbox.style использует данные из **сервиса** Mapbox, тогда для корректной работы необходимо будет зарегистрироваться на https://mapbox.com/ и бесплатно получить Mapbox Access токен (правда, придётся ввести данные кредитной карты). Этот токен затем должен быть передан в соответствующие параметры конфигурации. В противном случае регистрироваться и получать токен не требуется.

Значение, передаваемое в layout.mapbox.style, может быть одним из:

* "white-bg", что оставит просто белый лист как базу
* "open-street-map", "carto-positron", или "carto-darkmatter" - это карты, состоящие из растровых тайлов, причём их использование не требует регистрации или получение токена
* "basic", "streets", "outdoors", "light", "dark", "satellite", or "satellite-streets" - это карты из векторных тайлов, требуется регистрация и получение токена.
* ссылки на те или иные сервисы Mapbox, требуется токен.
* Mapbox Style объект (https://docs.mapbox.com/mapbox-gl-js/style-spec/)

In [ ]:
# нарисуем карту американских городов на OpenStreet
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv")

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="name", hover_data=["pop"],
                        color_discrete_sequence=["red"], zoom=4, height=400)
fig.update_layout(mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# нарисуем карту американских городов получив базовую карту по ссылке

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="name", hover_data=["pop"],
                        color_discrete_sequence=["red"], zoom=4, height=400)
fig.update_layout \
(
    mapbox_style="white-bg",
    mapbox_layers=\
    [{
        "below": 'traces', # под объектами данных
        "sourcetype": "raster", # растровые тайлы
        "sourceattribution": "United States Geological Survey",
        "source":
        [
            "https://basemap.nationalmap.gov/arcgis/rest/services/"
            "USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
        ]
    }]
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# нужен mapbox token
token = open(".mapbox_token").read()

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="name", hover_data=["pop"],
                        color_discrete_sequence=["red"], zoom=4, height=400)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# можно ограничивать угол обзора
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv")

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="name", hover_data=["pop"],
                        color_discrete_sequence=["red"], zoom=4, height=400)
fig.update_layout(mapbox_bounds={"west": -180, "east": -50, "south": 20, "north": 90})
fig.update_layout(mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

## 2. Фоновые карты (хороплеты)

[Фоновые карты или хороплеты](https://ru.wikipedia.org/wiki/Фоновая_картограмма) - это карты, в которых разным стилем или цветами показывают характеристики тех или иных территорий на ней.

При создании хороплета мы должны предоставить два главных типа вводных данных:

1. Информция о разметке:
    1. Это может быть соответствующий GeoJSON файл, где каждый атрибут имеет поле *id*, либо присутствует другая идентифицирующая объект величина, либо
    2. одна из встроенных в plotly разметок: штаты США или границы стран
2. Список значений, каждый из которых определен идентификатором.

Первые данные заносятся в аргумент *geojson*, а вторые - в аргумент *color* (в случае использования px.choropleth) или *z* (в случае использования объекта из graph_objects) в том же порядке, что и соответствующие ID в первом аргументе.

Также стоит отметить, что в аргумент *geojson* может быть передана ссылка на GeoJSON-файл

In [ ]:
# посмотрим как выглядят данные о разметке для стран:

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') \
as response:
    counties = json.load(response)

counties["features"][0]

In [ ]:
# данные по безработице для штатов

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                 dtype={"fips": str})
df.head()

In [ ]:
# строим

fig = px.choropleth\
(
    df,
    geojson=counties,
    locations='fips',
    color='unemp',
    color_continuous_scale="inferno",
    range_color=(0, 12),
    scope="usa",
    labels={'unemp':'unemployment rate'}
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# также можем воспользоваться mapbox

fig = px.choropleth_mapbox \
(
    df,
    geojson=counties,
    locations='fips',
    color='unemp',
    color_continuous_scale="inferno",
    range_color=(0, 12),
    mapbox_style="carto-positron",
    zoom=3,
    center={"lat": 37, "lon": -96},
    opacity=0.5,
    labels={'unemp':'unemployment rate'}
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# просмотрим данные голосования - заметим, что тут нет id, но есть district
df = px.data.election()
geojson = px.data.election_geojson()

print(df["district"][2])
print(geojson["features"][0]["properties"])

df

In [ ]:
# используем featureidkey
fig = px.choropleth\
(
    df,
    geojson=geojson,
    color="Joly",
    locations="district",
    featureidkey="properties.district",
    projection="mercator"
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# а теперь определим победителей в каждом округе
fig = px.choropleth_mapbox\
(
    df,
    geojson=geojson,
    color="winner",
    locations="district",
    featureidkey="properties.district",
    hover_data=["Bergeron", "Coderre", "Joly"],
    center={"lat": 45.6, "lon": -73.7},
    mapbox_style="carto-positron",
    zoom=9
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
# воспользуемся геопандасом

df = px.data.election()
geo_df = gpd.GeoDataFrame.from_features(px.data.election_geojson()["features"])
display(geo_df.head())
geo_df = geo_df.merge(df, on="district").set_index("district")
display(geo_df.head())

fig = px.choropleth_mapbox \
(
    geo_df,
    geojson=geo_df.geometry,
    locations=geo_df.index,
    color="Joly",
    center={"lat": 45.6, "lon": -73.7},
    mapbox_style="carto-positron",
    zoom=9
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
px.data.gapminder().query("year==2007")

In [ ]:
# воспользуемся локациями стран передав их коды в locations
# (так можно, так как локации стран встроены в plotly)
df = px.data.gapminder().query("year==2007")
fig = px.choropleth(df, locations="iso_alpha",
                    color="gdpPercap",
                    hover_name="country",
                    color_continuous_scale="solar")
fig.show()

In [ ]:
# то же самое прокатывает со штатами
fig = px.choropleth(locations=["NV", "TX", "MI"],
                    locationmode="USA-states",
                    scope="usa", color=["a", "b", "c"],
                    color_discrete_sequence=["cyan", "magenta", "yellow"])
fig.show()

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/unemployment_data_europe_oecd.csv")
df

In [ ]:
# то же самое можно делать и с помощью go.Chloropleth

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/unemployment_data_europe_oecd.csv")
df = df.query("TIME=='2023-01'")

fig = go.Figure\
(
    data=go.Choropleth\
    (
        locations=df['LOCATION'],
        z = df['Value'], # данные для цветового разделения
        locationmode = 'ISO-3', # метод для определения локации по коду
        colorscale = 'Reds',
        colorbar_title = "%"
    )
)

fig.update_layout\
(
    title_text = 'Uneployment rate in Europe in Jan 2023',
    geo_scope='europe',
    margin={"r": 0, "t": 35, "l": 0, "b": 0}
)

fig.show()

In [ ]:
fig = go.Figure \
(
    data=go.Choropleth \
    (
        locations=df['LOCATION'],
        z = df['Value'], # данные для цветового разделения
        locationmode = 'ISO-3', # метод для определения локации по коду
        colorscale = 'Reds',
        colorbar_title = "%"
    )
)

fig.update_layout \
(
    title_text = 'Uneployment rate in Europe in Jan 2023',
    margin={"r": 0, "t": 35, "l": 0, "b": 0}
)

fig.update_geos \
(
    scope='europe',
    showrivers=True,
    rivercolor='royalblue'
)

fig.show()

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/country_indicators.csv")
print(df["Indicator Name"].unique())

In [ ]:
df

In [ ]:
# сделаем мировую карту выбросов углекислоты

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/country_indicators.csv")
df = df.query("Year==2002 & `Indicator Name`=='CO2 emissions (metric tons per capita)'")

fig = go.Figure \
(
    data=go.Choropleth\
    (
        locations = df['Country Name'],
        locationmode = 'country names',
        z = df['Value'],
        text = df['Country Name'],
        colorscale = 'turbid',
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = 'tons per capita'
    )
)

fig.update_layout(
    title_text='2002 CO2 emissions (tons per capita)',
    geo=\
    {
        "showframe": False,
        "showcoastlines": False,
        "projection_type": 'equirectangular'
    }
)

fig.show()

In [ ]:
token = open(".mapbox_token").read()

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')\
as response:
    counties = json.load(response)

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})


fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df.fips, z=df.unemp,
                                    colorscale="Viridis", zmin=0, zmax=12, marker_line_width=0))
fig.update_layout(mapbox_style="light", mapbox_accesstoken=token,
                  mapbox_zoom=3, mapbox_center = {"lat": 37.1, "lon": -95.7})
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

## 3. Точечные графики на картах

In [ ]:
# стандартный пример - популяции стран

df = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo\
(
    df, locations="iso_alpha",
    size="pop",
)
fig.show()

In [ ]:
# обновление

fig = px.scatter_geo(df, locations="iso_alpha",
                     color="continent",
                     hover_name="country",
                     hover_data=["gdpPercap", "lifeExp"],
                     size="pop",
                     projection="natural earth")
fig.show()

In [ ]:
# можно и с mapbox
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv")

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="name", size="pop", hover_data=["pop"],
                        color_discrete_sequence=["red"], zoom=3, height=400)
fig.update_layout(mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/'
                 'master/2011_february_us_airport_traffic.csv')
df

In [ ]:
# выведем данные для американских аэропортов

df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)

fig = go.Figure\
(
    data=go.Scattergeo\
    (
        lon = df['long'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        marker_color = df['cnt'],
    )
)

fig.update_layout\
(
    title = 'Most trafficked US airports<br>(Hover for airport names)',
    geo_scope='usa',
)
fig.show()

In [ ]:
# теперь можно их стилизовать

fig = go.Figure \
(
    data=go.Scattergeo\
    (
        lon = df['long'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        marker = \
        {
            "size": 6,
            "opacity": 0.75,
            "symbol": 'diamond',
            "line": \
            {
                "width": 2,
                "color": 'grey'
            },
            "colorscale": 'Reds',
            "cmin": 0,
            "color": df['cnt'],
            "cmax": df['cnt'].max(),
            "colorbar_title":"Incoming flights<br>February 2011"
        }
    )
)


fig.update_layout\
(
    title = 'Most trafficked US airports<br>(Hover for airport names)',
    geo = \
    {
        "scope":'usa',
        "showland": True,
        "subunitwidth": 0.4
    }
)
fig.show()

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2015_06_30_precipitation.csv')
df

In [ ]:
# данные по осадкам



fig = go.Figure \
(
    data=go.Scattergeo \
    (
        lat = df['Lat'],
        lon = df['Lon'],
        text = df['Globvalue'].astype(str) + ' inches',
        marker = \
        {
            "color": df['Globvalue'],
            "colorscale": "Teal",
            "opacity": 0.7,
            "size": 2,
            "colorbar": \
            {
                "titleside": "right",
                "ticks": "outside",
                "dtick": 0.1
            }
        }
    )
)

fig.update_layout(
    geo = \
    {
        "scope": 'north america',
        "showcoastlines": True, "coastlinecolor": "RebeccaPurple",
        "showland": True, "landcolor": "green",
        "showlakes": True, "lakecolor": "aqua",
        "showrivers": True, "rivercolor": "royalblue",
        "subunitcolor": "Black",
        "countrycolor": "Black",
        "showsubunits": True,
        "showcountries": True,
        "resolution": 50,
        "projection": \
        {
            "type": 'conic conformal',
            "rotation_lon": -100
        },
        "lonaxis": \
        {
            "showgrid": True,
            "gridwidth": 0.5,
            "range": [-140.0, -55.0],
            "dtick": 5
        },
        "lataxis": \
        {
            "showgrid": True,
            "gridwidth": 0.5,
            "range": [20.0, 60.0],
            "dtick": 5
        }
    },
    title='US Precipitation 06-30-2015<br>Source: <a href="http://water.weather.gov/precip/">NOAA</a>',
)
fig.show()

In [ ]:
# можно заменять маркеры на иконки

token = open(".mapbox_token").read() # you need your own token

fig = go.Figure\
(
    go.Scattermapbox\
    (
        mode="markers+text+lines",
        lon=[-75, -80, -50],
        lat=[45, 20, -20],
        marker={'size': 20, 'symbol': ["bus", "harbor", "airport"]},
        text=["Bus", "Harbor", "airport"],
        textposition="bottom right"
    )
)

fig.update_layout\
(
    mapbox = \
    {
        'accesstoken': token,
        'style': "outdoors",
        'zoom': 0.7
    },
    showlegend = False)

fig.show()

In [ ]:
# можно кластеризовать маркеры, которые будут раскрываться по ходу дела

px.set_mapbox_access_token(open(".mapbox_token").read())
df = pd.read_csv \
(
    "https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv"
)
fig = px.scatter_mapbox(df, lat="lat", lon="long", size="cnt", zoom=3)
fig.update_traces(cluster={"enabled": True})
fig.show()

## 4. Линии на картах

In [ ]:
df = px.data.gapminder().query("year == 2007")
fig = px.line_geo(df, locations="iso_alpha",
                  color="continent",
                  projection="orthographic")
fig.show()

In [ ]:
# здесь мы распакуем все точки линий рек, чтобы выводилась аннотация при наведении на каждую из них

# скачиваем файл
wget.download("https://plotly.github.io/datasets/ne_50m_rivers_lake_centerlines.zip")

# распаковываем файл с точками рек
geo_df = gpd.read_file("zip://ne_50m_rivers_lake_centerlines.zip")

lats = []
lons = []
names = []

display(geo_df)

# в цикле проходимся по всем формам рек и экстрактируем нужные нам атрибуты
for feature, name in zip(geo_df.geometry, geo_df.name):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)

# таким образом получаем всё что нужно
fig = px.line_geo(lat=lats, lon=lons, hover_name=names)
fig.show()

In [ ]:
# можно в mapbox

fig = px.line_mapbox(lat=lats, lon=lons, hover_name=names,
                     mapbox_style="open-street-map", zoom=1)
fig.show()

In [ ]:
df_flight_paths.head()

In [ ]:
# сделаем визуализацию с перелётами между аэропортами

df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/'
                          'master/2011_february_us_airport_traffic.csv')
df_airports.head()

df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/'
                              'master/2011_february_aa_flight_paths.csv')
df_flight_paths.head()

fig = go.Figure()

# добавляем маркеры аэропортов
fig.add_trace \
(
    go.Scattergeo \
    (
        locationmode = 'USA-states',
        lon = df_airports['long'],
        lat = df_airports['lat'],
        hoverinfo = 'text',
        text = df_airports['airport'],
        mode = 'markers',
        marker = \
        {
            "size": 2,
            "color": 'red',
            "line": \
            {
                "width": 2,
                "color": 'grey'
            }
        }
    )
)

# добавляем линии перелётов с интенсивностью цвета в зависимости от популярности направления
flight_paths = []
for i in range(len(df_flight_paths)):
    fig.add_trace \
    (
        go.Scattergeo \
        (
            locationmode = 'USA-states',
            lon = [df_flight_paths['start_lon'][i], df_flight_paths['end_lon'][i]],
            lat = [df_flight_paths['start_lat'][i], df_flight_paths['end_lat'][i]],
            mode = 'lines',
            line = {"width": 1, "color": 'red'},
            opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
        )
    )

# обновляем дизайн
fig.update_layout\
(
    title_text = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
    showlegend = False,
    geo = \
    {
        "scope": 'north america',
        "projection_type": 'azimuthal equal area',
        "showland": True,
        "landcolor": 'azure',
        "countrycolor": 'lightsteelblue',
    },
)

fig.show()

In [ ]:
# чтобы улучшить перформанс, нужно объединить все линии в один график:

fig = go.Figure()

# добавляем маркеры аэропортов
fig.add_trace \
(
    go.Scattergeo \
    (
        locationmode = 'USA-states',
        lon = df_airports['long'],
        lat = df_airports['lat'],
        hoverinfo = 'text',
        text = df_airports['airport'],
        mode = 'markers',
        marker = \
        {
            "size": 2,
            "color": 'red',
            "line": \
            {
                "width": 2,
                "color": 'grey'
            }
        }
    )
)

# здесь происходит оптимизация
lons = []
lats = []
lons = np.empty(3 * len(df_flight_paths))
lons[::3] = df_flight_paths['start_lon']
lons[1::3] = df_flight_paths['end_lon']
lons[2::3] = None # тут вставляем как бы разрыв между линиями
lats = np.empty(3 * len(df_flight_paths))
lats[::3] = df_flight_paths['start_lat']
lats[1::3] = df_flight_paths['end_lat']
lats[2::3] = None # и тут тоже

# теперь добавляем все линии как сущность одного графика
fig.add_trace\
(
    go.Scattergeo\
    (
        locationmode = 'USA-states',
        lon = lons,
        lat = lats,
        mode = 'lines',
        line = {"width": 1, "color": 'red'},
        opacity = 0.5
    )
)

# обновляем дизайн
fig.update_layout\
(
    title_text = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
    showlegend = False,
    geo = \
    {
        "scope": 'north america',
        "projection_type": 'azimuthal equal area',
        "showland": True,
        "landcolor": 'azure',
        "countrycolor": 'lightsteelblue',
    },
)

fig.show()

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/globe_contours.csv')
df.head()

In [ ]:
# рисуем линии на ортографической проекции шара

fig = go.Figure()


# добавляем линии
for i, (lat, lon) in enumerate(zip(df.columns[::2], df.columns[1::2])):
    fig.add_trace \
    (
        go.Scattergeo \
        (
            lon=df[lon],
            lat=df[lat],
            mode='lines',
            line={"width": 2, "color": px.colors.diverging.RdBu[i%len(px.colors.diverging.RdBu)]}
        )
    )

# обновляем внешний вид
fig.update_layout(
    title_text='Contour lines over globe<br>(Click and drag to rotate)',
    showlegend=False,
    geo = \
    {
        "scope": 'north america',
        "showcoastlines": True, "coastlinecolor": "RebeccaPurple",
        "showland": True, "landcolor": "green",
        "showlakes": True, "lakecolor": "aqua",
        "showrivers": True, "rivercolor": "royalblue",
        "subunitcolor": "Black",
        "countrycolor": "Black",
        "showsubunits": True,
        "showcountries": True,
        "resolution": 50,
        "projection": \
        {
            "type": 'orthographic',
            "rotation": {"lon": -100, "lat": 40}
        },
        "lonaxis": \
        {
            "showgrid": True,
            "gridwidth": 0.5,
        },
        "lataxis": \
        {
            "showgrid": True,
            "gridwidth": 0.5,
        }
    },
)

fig.show()

In [ ]:
# также можно добавлять линии с помощью scattermapbox

fig = go.Figure\
(
    go.Scattermapbox\
    (
        mode="markers+lines",
        lon=[37.618, 30.304, 49.109],
        lat=[55.751, 59.938, 55.796],
        marker={'size': 10}
    )
)

fig.add_trace\
(
    go.Scattermapbox\
    (
        mode="markers+lines",
        lon=[37.618, 135.067, 131.87353],
        lat=[55.751, 48.466, 43.10562],
        marker={'size': 10}\
    )
)

fig.update_layout\
(
    margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
    mapbox=\
    {
        'style': "open-street-map",
        'center': {'lon': 80, 'lat': 55},
        'zoom': 2
    }
)

fig.show()

## 5. Остальные возможности plotly

### Пузырьковая карта

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/unemployment_data_europe_oecd.csv")
df

In [ ]:
df["TIME"] = pd.to_datetime(df["TIME"])

In [ ]:
# иллюстрация безработицы

fig = px.scatter_geo(df[df["TIME"].dt.year == 2003], locations="LOCATION",
                     hover_name="LOCATION", size="Value",
                     projection="natural earth")
fig.show()

In [ ]:
# анимация безработицы
fig = px.scatter_geo(df, locations="LOCATION",
                     hover_name="LOCATION", size="Value",
                     animation_frame="TIME",
                     projection="natural earth")
fig.show()

In [ ]:
# выводим топ городов по населению

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_us_cities.csv')
df.head()

df['text'] = df['name'] + '<br>Population ' + (df['pop']/1e6).astype(str)+' million'
limits = [(0, 2), (3, 10), (11, 20), (21, 50), (50, 3000)]
colors = ["royalblue", "crimson", "lightseagreen", "orange", "lightgrey"]
cities = []
scale = 5000

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    fig.add_trace\
    (
        go.Scattergeo\
        (
            locationmode = 'USA-states',
            lon = df_sub['lon'],
            lat = df_sub['lat'],
            text = df_sub['text'],
            marker = \
            {
                "size": df_sub['pop']/scale,
                "color": colors[i],
                "line_color": 'grey',
                "line_width": 0.5,
                "sizemode": 'area'
            },
            name='{0} - {1}'.format(lim[0] + 1, lim[1] + 1)
        )
    )

fig.update_layout\
(
        title_text = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = \
        {
            "scope": 'usa',
            "landcolor": 'black'
        }
)

fig.show()

### Карта плотности

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/earthquakes-23k.csv')
df

In [ ]:
fig = px.density_mapbox(df, lat='Latitude', lon='Longitude', z='Magnitude', radius=7,
                        center={"lat": 0, "lon": 100}, zoom=0,
                        mapbox_style="open-street-map")
fig.show()

In [ ]:
# можно сделать с помощью Densitymapbox

fig = go.Figure\
(
    go.Densitymapbox\
    (
        lat=df.Latitude,
        lon=df.Longitude,
        z=df.Magnitude,
        radius=7
    )
)
fig.update_layout(mapbox_style="open-street-map", mapbox_center_lon=100, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Дополнительные материалы

[Зарекистрироваться и получить Mapbox Access токен можно здесь](https://account.mapbox.com/auth/signup/)

[Как создавать закрашенные зоны на картах](https://plotly.com/python/filled-area-on-mapbox/)

[Ячеистые диаграммы на картах](https://plotly.com/python/hexbin-mapbox/)

[Документация по layout.geo](https://plotly.com/python/reference/layout/geo/)

[Документация по layout.mapbox](https://plotly.com/python/reference/layout/mapbox/)